In [12]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from dateutil import parser
from datetime import datetime



#initiate data storage
Date = []
Author = []
Title = []
Articles = []
PDFlink = []
df = pd.DataFrame(columns=["Date","Author","Title","Articles"])
href='<a href="publications/news/774001_22092020-1.pdf"'
start = href.find('<a href="')
for page in range(1,5):
        r = requests.get("http://www.bnacapitaux.com.tn/?pg="+str(page)+"&news=all")
        soup =   BeautifulSoup(r.content,"lxml")
        #Extract date
        for dateDiv in soup.find_all(class_="NewsDate")[:6]:
            match = re.search(r'\d{2}/\d{2}/\d{4}', dateDiv.text.replace("\n",""))
            datetime = parser.parse(match.group())
            Date.append(datetime)
        
        #Extract author
        for author in soup.find_all(class_="NewsDate")[:6]:
            
                Author.append(author.text.replace("\n","").strip()[25:-1])
            
                
        #Extract title       
        for title in soup.find_all(class_="NewsTitle")[:6]:
            Title.append(title.text.replace("\n","").strip())
            
        #Extract description    
        for article in soup.find_all('div', id=lambda x: x and x.startswith('NEW_DESC'))[:6]:
            
                Articles.append(article.text.replace("\n","").strip())
                
        #Extract PDFlink        
        liste=[]
        liste1=[]
        for link in soup.find_all('a', href=lambda x: x and x.startswith('publications')):
            match = re.search(r'\bAttachement\b', str(link))
            if match !=None :
                liste.append(str(link)[9:60])
                for l in liste:
                    pos= l.find('"')
                    l=("http://www.bnacapitaux.com.tn/"+l[0:pos])
                liste1.append(l)     
        PDFlink=PDFlink+liste1[0:6]

#Fill the dataframe
for i in range(len(Date)):
    df = df.append({'Date':Date[i],'Author':Author[i],'Title':Title[i],'Articles':Articles[i],'PDF':PDFlink[i]},ignore_index=True)

In [17]:
df.head()

,Date,Author,Title,Articles,PDF
0,2020-10-20,CITY CARS,Indicateurs d'activité arrêtés au 30/09/2020,Le concessionnaire de la marque KIA a réalisé ...,http://www.bnacapitaux.com.tn/publications/new...
1,2020-10-20,CARTHAGE CEMENT,Indicateurs d'activité arrêtés au 30/09/2020,Le chiffre d’affaires réalisé courant les neuf...,http://www.bnacapitaux.com.tn/publications/new...
2,2020-10-20,UNIMED,Indicateurs d'activité arrêtés au 30/09/2020,Le chiffre d’affaires UNIMED au 30 Septembre 2...,http://www.bnacapitaux.com.tn/publications/new...
3,2020-10-20,TUNISIE LEASING F,Indicateurs d'activité arrêtés au 30/09/2020,"Une régression des mises en force (204,793MD) ...",http://www.bnacapitaux.com.tn/publications/new...
4,2020-10-20,CELLCOM,Indicateurs d'activité arrêtés au 30/09/2020,Le chiffre d’affaires au 30/09/2020 s’élève...,http://www.bnacapitaux.com.tn/publications/new...


In [33]:
df.to_excel(r'./ArticlesBNAcapitaux.xlsx', index = False)